> inspired from https://medium.com/nightingale/how-to-create-eye-catching-maps-with-python-and-kepler-gl-e7e897eff8ac

# earthquakes

In [1]:
import pandas as pd
import keplergl 

/Users/tparment/miniconda3/envs/flotpython-slides/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## data acquisition

In [2]:
# could not spot a direct URL to this data...
dataset_url = "https://catalog.data.gov/dataset/global-significant-earthquake-database-2150-bc-to-present"

# so I copied it in this repo instead
dataset_filename = "Worldwide-Earthquake-database.csv"

In [3]:
%cd raw-tps/keplergl

/Users/tparment/git/flotpython-slides/raw-tps/keplergl


In [4]:
df = pd.read_csv(dataset_filename)
df.describe()

,I_D,YEAR,MONTH,DAY,HOUR,MINUTE,FOCAL_DEPTH,EQ_PRIMARY,EQ_MAG_MW,EQ_MAG_MS,...,TOTAL_MISSING,TOTAL_MISSING_DESCRIPTION,TOTAL_INJURIES,TOTAL_INJURIES_DESCRIPTION,TOTAL_DAMAGE_MILLIONS_DOLLARS,TOTAL_DAMAGE_DESCRIPTION,TOTAL_HOUSES_DESTROYED,TOTAL_HOUSES_DESTROYED_DESCRIPTION,TOTAL_HOUSES_DAMAGED,TOTAL_HOUSES_DAMAGED_DESCRIPTION
count,6193.000000,6193.000000,5786.000000,5632.000000,4151.000000,3946.000000,3228.000000,4402.000000,1321.000000,2928.000000,...,25.00000,26.000000,1253.000000,1433.000000,453.000000,3273.000000,8.130000e+02,1777.000000,4.210000e+02,809.000000
mean,4771.385112,1805.694009,6.508296,15.741655,11.303060,28.820578,41.123916,6.459223,6.513475,6.574761,...,1910.68000,1.807692,2390.834796,1.951849,1904.812208,2.204400,1.828339e+04,2.697243,5.985086e+04,2.464771
std,3004.473731,375.314913,3.451206,8.751377,7.031131,17.153131,70.448975,1.044992,0.929940,0.990647,...,8667.79685,1.059027,27489.582539,1.079804,12509.939103,1.039891,1.955296e+05,1.051896,1.023722e+06,1.131921
min,1.000000,-2150.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.600000,3.600000,2.100000,...,1.00000,1.000000,1.000000,1.000000,0.010000,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000
25%,2186.000000,1820.000000,4.000000,8.000000,5.000000,14.000000,10.000000,5.700000,5.800000,5.800000,...,5.00000,1.000000,10.000000,1.000000,5.000000,1.000000,6.300000e+01,2.000000,9.000000e+01,1.000000
50%,4688.000000,1928.000000,7.000000,16.000000,11.000000,29.000000,25.000000,6.500000,6.500000,6.600000,...,21.00000,1.000000,41.000000,1.000000,30.000000,2.000000,5.060000e+02,3.000000,6.760000e+02,3.000000
75%,6589.000000,1988.000000,9.000000,23.000000,17.000000,43.750000,40.000000,7.300000,7.200000,7.300000,...,138.00000,3.000000,200.000000,3.000000,300.000000,3.000000,3.600000e+03,3.000000,3.000000e+03,3.000000
max,10501.000000,2020.000000,12.000000,31.000000,23.000000,59.000000,675.000000,9.500000,9.500000,9.100000,...,43476.00000,4.000000,799000.000000,4.000000,220085.456000,4.000000,5.360000e+06,4.000000,2.100000e+07,4.000000


## clean up

In [5]:
# lat and lon to numeric, errors converted to nan
df['LONGITUDE'] = pd.to_numeric(df.LONGITUDE, errors='coerce')
df['LATITUDE'] = pd.to_numeric(df.LATITUDE, errors='coerce')

In [6]:
# drop rows with missing lat, lon, and intensity
df.dropna(subset=['LONGITUDE', 'LATITUDE', 'INTENSITY'], inplace=True)

In [7]:
# convert tsunami flag from string to int
df['FLAG_TSUNAMI'] = [1 if i=='Yes' else 0 for i in df.FLAG_TSUNAMI.values]

## draw map

In [8]:
kepler_map = keplergl.KeplerGl(height=600)
kepler_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(height=600)

In [9]:
kepler_map.add_data(data=df, name="earthquakes")

/Users/tparment/miniconda3/envs/flotpython-slides/lib/python3.9/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


## config

In [10]:
manual_config = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "o2783kd",
          "type": "point",
          "config": {
            "dataId": "earthquakes",
            "label": "earthquakes",
            "color": [
              23,
              184,
              190
            ],
            "columns": {
              "lat": "LATITUDE",
              "lng": "LONGITUDE",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Uber Viz Diverging 0",
                "type": "diverging",
                "category": "Uber",
                "colors": [
                  "#C22E00",
                  "#FEEEE8",
                  "#00939C"
                ],
                "reversed": True
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                2,
                12
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "FLAG_TSUNAMI",
              "type": "integer"
            },
            "colorScale": "quantize",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": {
              "name": "INTENSITY",
              "type": "integer"
            },
            "sizeScale": "sqrt"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "earthquakes": [
              "I_D",
              "FLAG_TSUNAMI",
              "YEAR",
              "MONTH",
              "DAY"
            ]
          },
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 37.48434368318514,
      "longitude": -122.11886458964356,
      "pitch": 0,
      "zoom": 8.198674684017652,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

# graffiti

***this part needs more work***

In [11]:
import pandas as pd
import geopandas as gpd
import keplergl 

In [12]:
block_url = "https://opendata.vancouver.ca/explore/dataset/block-outlines/download/?format=geojson&timezone=Europe/Berlin&lang=en"
block_filename = "block.geojson"

In [13]:
graffiti_url = "https://opendata.vancouver.ca/explore/dataset/graffiti/download/?format=geojson&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B"
graffiti_filename = "graffiti.geojson"

In [14]:
import requests
def store_url_as_filename(url, filename, force=False):
    from pathlib import Path
    if force or not Path(filename).exists():
        req = requests.get(url)
        text = req.text
        with open(filename, 'w') as writer:
            writer.write(text)

In [15]:
store_url_as_filename(block_url, block_filename)
store_url_as_filename(graffiti_url, graffiti_filename)

In [16]:
df_block = gpd.read_file(block_filename)
df_block.dropna(inplace=True)
df_block.head()

,geometry
0,"POLYGON ((-123.03445 49.27218, -123.03459 49.2..."
1,"POLYGON ((-123.03906 49.27184, -123.03905 49.2..."
2,"POLYGON ((-123.04362 49.27219, -123.04396 49.2..."
3,"POLYGON ((-123.03444 49.27311, -123.03458 49.2..."
4,"POLYGON ((-123.03652 49.27277, -123.03651 49.2..."


In [17]:
df_graffiti = gpd.read_file(graffiti_filename)
df_graffiti.dropna(inplace=True)
df_graffiti.head()

,count,geo_local_area,geometry
0,2,Kitsilano,POINT (-123.16881 49.25802)
1,2,Kitsilano,POINT (-123.16897 49.25802)
2,1,Grandview-Woodland,POINT (-123.07478 49.27826)
3,2,Kensington-Cedar Cottage,POINT (-123.07678 49.25087)
4,1,Renfrew-Collingwood,POINT (-123.05306 49.23992)


In [18]:
# join datasets
df = gpd.sjoin(df_block, df_graffiti, how='inner', predicate='contains')

In [19]:
# create new indexes
df.reset_index(inplace=True)
df.head()

,index,geometry,index_right,count,geo_local_area
0,7,"POLYGON ((-123.02612 49.22584, -123.02611 49.2...",4487,1,Killarney
1,7,"POLYGON ((-123.02612 49.22584, -123.02611 49.2...",369,1,Killarney
2,7,"POLYGON ((-123.02612 49.22584, -123.02611 49.2...",809,1,Killarney
3,8,"POLYGON ((-123.04350 49.26933, -123.04401 49.2...",602,2,Hastings-Sunrise
4,8,"POLYGON ((-123.04350 49.26933, -123.04401 49.2...",115,1,Hastings-Sunrise
